##### Это финальное задание по курсу «Обучение на размеченных данных».

В нем вы сравните логистическую регрессию и случайный лес на разных наборах признаков. В качестве данных будет использован Adult Data Set из репозитория UCI. В нем нужно предсказать, получает ли человек больше 50 000$ в год, или нет, по ряду признаков, таких как пол, образование, раса и др. Подробное описание можно найти по ссылке: https://archive.ics.uci.edu/ml/datasets/Adult.

In [107]:
import numpy as np
import pandas as pd

### Загрузка данных

В ходе задания будем использовать __train_data.csv__ для обучения моделей, на нем же и будем производить кросс-валидацию. В качестве отложенной выборки будем использовать __test_data.csv__

In [108]:
column_names = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
                "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
                "hours-per-week", "native-country", "class"]

In [109]:
train = pd.read_csv("train_data.csv", sep=", ", header=None, engine="python", names=column_names)

In [110]:
test = pd.read_csv("test_data.csv", sep=", ", header=None, engine="python", names=column_names)

### 1. Пропущенные значения 

В обучающей выборке порядка 7% строк имеют пропущенные значения (вместо значения поля указан вопросительный знак __'?'__). В каких признаках в обучающей и тестовой выборке имеются пропущенные значения? Так как они все пропущены в категориальных признаках, то можно пока их ни на что не заменять, а просто считать еще одной категорией.

In [111]:
black_list_test=[]
lis=[]
for col in test.columns:
    b=0
    if test[col].dtypes!='int64':
        lis=test[col].str.find('?').tolist()
        for i in range(len(lis)):
            if lis[i]==0:
                b=1
        if b==1:
            black_list_test.append(col)
print('В этих столбцах тестовой выборки есть пропущенные значения:')
print(black_list_test)

В этих столбцах тестовой выборки есть пропущенные значения:
['workclass', 'occupation', 'native-country']


In [112]:
black_list_train=[]
lis=[]
for col in train.columns:
    b=0
    if train[col].dtypes!='int64':
        lis=train[col].str.find('?').tolist()
        for i in range(len(lis)):
            if lis[i]==0:
                b=1
        if b==1:
            black_list_train.append(col)
print('В этих столбцах обучающей выборки есть пропущенные значения:')
print(black_list_train)

В этих столбцах обучающей выборки есть пропущенные значения:
['workclass', 'occupation', 'native-country']


### 2. Обучение на вещественных признаках

В этом разделе обучите модели только на вещественных признаках ("continuous" в описании данных). Обучите логистическую регрессию (linear_model.LogisticRegression) и случайный лес (ensemble.RandomForestClassifier) из sklearn. В первом случае подберите оптимальные параметры $penalty$ и $C$ на отрезке $[10^{-6}, 10^{6}]$ (по степеням $10$ с шагом $1$, начиная с $-6$), а во втором при фиксированном числе деревьев в 50 подберите $max\_depth$ и $min\_samples\_split$ из отрезка $[2, 14]$ с шагом в 2 и множества $\{1, 2, 4, 8\}$ соответственно. За целевую метрику качества возьмите AUC-ROC. В качестве схемы валидации используйте стратифицированную кросс-валидацию по 5-ти фолдам. Какие параметры оказались оптимальными?

Учтите, что целевая переменная в датасете является строкой. Поэтому для начала ее нужно перевести в бинарную величину. Также не забудьте отмасштабировать данные с помощью StandartScaler'а из модуля preprocessing.

In [200]:
y_train=train['class']
X_train=train.drop(['class'], axis=1)
y_test=test['class']
X_test=test.drop(['class'], axis=1)

numeric_cols = ['age','fnlwgt','education-num','capital-gain','capital-loss','hours-per-week']
categorical_cols = list(set(column_names) - set(numeric_cols))
categorical_cols.remove('class')

y1=y_train.replace('<=50K',1)
y_train_bin=y1.replace('>50K',0)
y2=y_test.replace('<=50K',1)
y_test_bin=y2.replace('>50K',0)

from sklearn.preprocessing import StandardScaler
scl=StandardScaler()
X_train_num_scaled=scl.fit_transform(X_train[numeric_cols])
X_test_num_scaled=scl.transform(X_test[numeric_cols])

In [201]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn import ensemble
from sklearn import metrics
param_grid={'C': [10**(-6),10**(-5),10**(-4),10**(-3),10**(-2),10**(-1),10**0,10**1,10**2,10**3,10**4,10**5,10**6],'penalty':['l1', 'l2']}
optimizer = GridSearchCV(LogisticRegression(solver='liblinear'),param_grid,cv=5, scoring='roc_auc')
param_grid_rf = {'max_depth':[2,4,6,8,10,12,14],'min_samples_split':[2,4,8]}
optimizer_rf = GridSearchCV(ensemble.RandomForestClassifier(n_estimators = 50),param_grid_rf,cv=5, scoring='roc_auc')

In [202]:
optimizer.fit(X_train_num_scaled, y_train_bin)
optimizer_rf.fit(X_train_num_scaled, y_train_bin)
print ('Лучшие параметры для модели LogisticRegression: {}'.format(optimizer.best_params_))
print ('Лучшие параметры для ансамбля RandomForestClassifier: {}'.format(optimizer_rf.best_params_))

Лучшие параметры для модели LogisticRegression: {'C': 0.1, 'penalty': 'l2'}
Лучшие параметры для ансамбля RandomForestClassifier: {'max_depth': 12, 'min_samples_split': 2}


Посчитайте accuracy, precision, recall, f1-score и AUC-ROC на отложенной выборке для оптимальных алгоритмов. Какие они получились? Какой алгоритм лучше?

In [203]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score 
print('Доля верных ответов согласно метрике AUC ROC для модели LogisticRegression: {} '.format(roc_auc_score(y_test_bin,optimizer.best_estimator_.predict_proba(X_test_num_scaled)[:, 1])))
print('Доля верных ответов согласно метрике AUC ROC для ансамбля RandomForestClassifier: {} '.format(roc_auc_score(y_test_bin,optimizer_rf.best_estimator_.predict_proba(X_test_num_scaled)[:, 1])))
print('Доля верных ответов согласно метрике precision для модели LogisticRegression: {} '.format(precision_score(y_test_bin,optimizer.best_estimator_.predict(X_test_num_scaled))))
print('Доля верных ответов согласно метрике precision для ансамбля RandomForestClassifier: {} '.format(precision_score(y_test_bin,optimizer_rf.best_estimator_.predict(X_test_num_scaled))))
print('Доля верных ответов согласно метрике recall для модели LogisticRegression: {} '.format(recall_score(y_test_bin,optimizer.best_estimator_.predict(X_test_num_scaled))))
print('Доля верных ответов согласно метрике recall для ансамбля RandomForestClassifier: {} '.format(recall_score(y_test_bin,optimizer_rf.best_estimator_.predict(X_test_num_scaled))))
print('Доля верных ответов согласно метрике accuracy для модели LogisticRegression: {} '.format(accuracy_score(y_test_bin,optimizer.best_estimator_.predict(X_test_num_scaled))))
print('Доля верных ответов согласно метрике accuracy для ансамбля RandomForestClassifier: {} '.format(accuracy_score(y_test_bin,optimizer_rf.best_estimator_.predict(X_test_num_scaled))))
print('Доля верных ответов согласно метрике f1-score для модели LogisticRegression: {} '.format(f1_score(y_test_bin,optimizer.best_estimator_.predict(X_test_num_scaled))))
print('Доля верных ответов согласно метрике f1-score для ансамбля RandomForestClassifier: {} '.format(f1_score(y_test_bin,optimizer_rf.best_estimator_.predict(X_test_num_scaled))))

Доля верных ответов согласно метрике AUC ROC для модели LogisticRegression: 0.8255017510712491 
Доля верных ответов согласно метрике AUC ROC для ансамбля RandomForestClassifier: 0.8608481733720496 
Доля верных ответов согласно метрике precision для модели LogisticRegression: 0.83241330502477 
Доля верных ответов согласно метрике precision для ансамбля RandomForestClassifier: 0.8482714468629962 
Доля верных ответов согласно метрике recall для модели LogisticRegression: 0.9458785685564938 
Доля верных ответов согласно метрике recall для ансамбля RandomForestClassifier: 0.9589867310012062 
Доля верных ответов согласно метрике accuracy для модели LogisticRegression: 0.8132178613107303 
Доля верных ответов согласно метрике accuracy для ансамбля RandomForestClassifier: 0.8376635341809471 
Доля верных ответов согласно метрике f1-score для модели LogisticRegression: 0.8855260681347639 
Доля верных ответов согласно метрике f1-score для ансамбля RandomForestClassifier: 0.9002377986637979 


Однозначно, алгоритм RandomForestClassifier показал себя лучше, чем LogisticRegression

### 3. Категориальные признаки как есть

Теперь к вещественным добавьте категориальные признаки, заменив их на числа с помощью LabelEncoder из модуля preprocessing. Переподберите параметры для логистической регрессии и случайного леса аналогично прошлому пункту. Как изменилось качество моделей на тестовой выборке? Как вы можете это объяснить?

In [219]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
X_train_cat=X_train[categorical_cols].apply(lambda x: le.fit_transform(x))
X_test_cat=X_test[categorical_cols].apply(lambda x: le.fit_transform(x))
X_train_full=np.hstack((X_train_num_scaled,X_train_cat))
X_test_full=np.hstack((X_test_num_scaled,X_test_cat))

In [221]:
optimizer.fit(X_train_full, y_train_bin)
optimizer_rf.fit(X_train_full, y_train_bin)
print ('Лучшие параметры для модели LogisticRegression для всех данных: {}'.format(optimizer.best_params_))
print ('Лучшие параметры для ансамбля RandomForestClassifier для всех данных: {}'.format(optimizer_rf.best_params_))

Лучшие параметры для модели LogisticRegression для всех данных: {'C': 1, 'penalty': 'l2'}
Лучшие параметры для ансамбля RandomForestClassifier для всех данных: {'max_depth': 14, 'min_samples_split': 4}


In [223]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score 
print('Доля верных ответов согласно метрике AUC ROC для модели LogisticRegression: {} '.format(roc_auc_score(y_test_bin,optimizer.best_estimator_.predict_proba(X_test_full)[:, 1])))
print('Доля верных ответов согласно метрике AUC ROC для ансамбля RandomForestClassifier: {} '.format(roc_auc_score(y_test_bin,optimizer_rf.best_estimator_.predict_proba(X_test_full)[:, 1])))
print('Доля верных ответов согласно метрике precision для модели LogisticRegression: {} '.format(precision_score(y_test_bin,optimizer.best_estimator_.predict(X_test_full))))
print('Доля верных ответов согласно метрике precision для ансамбля RandomForestClassifier: {} '.format(precision_score(y_test_bin,optimizer_rf.best_estimator_.predict(X_test_full))))
print('Доля верных ответов согласно метрике recall для модели LogisticRegression: {} '.format(recall_score(y_test_bin,optimizer.best_estimator_.predict(X_test_full))))
print('Доля верных ответов согласно метрике recall для ансамбля RandomForestClassifier: {} '.format(recall_score(y_test_bin,optimizer_rf.best_estimator_.predict(X_test_full))))
print('Доля верных ответов согласно метрике accuracy для модели LogisticRegression: {} '.format(accuracy_score(y_test_bin,optimizer.best_estimator_.predict(X_test_full))))
print('Доля верных ответов согласно метрике accuracy для ансамбля RandomForestClassifier: {} '.format(accuracy_score(y_test_bin,optimizer_rf.best_estimator_.predict(X_test_full))))
print('Доля верных ответов согласно метрике f1-score для модели LogisticRegression: {} '.format(f1_score(y_test_bin,optimizer.best_estimator_.predict(X_test_full))))
print('Доля верных ответов согласно метрике f1-score для ансамбля RandomForestClassifier: {} '.format(f1_score(y_test_bin,optimizer_rf.best_estimator_.predict(X_test_full))))

Доля верных ответов согласно метрике AUC ROC для модели LogisticRegression: 0.8509542705793476 
Доля верных ответов согласно метрике AUC ROC для ансамбля RandomForestClassifier: 0.915411789772757 
Доля верных ответов согласно метрике precision для модели LogisticRegression: 0.8462039045553146 
Доля верных ответов согласно метрике precision для ансамбля RandomForestClassifier: 0.8813204869669131 
Доля верных ответов согласно метрике recall для модели LogisticRegression: 0.9411338962605549 
Доля верных ответов согласно метрике recall для ансамбля RandomForestClassifier: 0.9489344591877764 
Доля верных ответов согласно метрике accuracy для модели LogisticRegression: 0.8243965358393219 
Доля верных ответов согласно метрике accuracy для ансамбля RandomForestClassifier: 0.8633990541121552 
Доля верных ответов согласно метрике f1-score для модели LogisticRegression: 0.8911479154768702 
Доля верных ответов согласно метрике f1-score для ансамбля RandomForestClassifier: 0.9138785625774473 


Качество моделей улучшилось благодаря добавлению категориальных признаков. Скорее всего так произошло потому, что среди категориальных признаков были те, которые лучше давали характеристику классу

### 4. Бинарное кодирование категориальных признаков

А теперь замените категориальные признаки из прошлого пункта на бинарно закодированные. Опять переподберите параметры для моделей и проверьте качество на тестовой выборке. Как изменилось качество относительно предыдущего пункта? Как вы можете это объяснить?

In [282]:
from sklearn.feature_extraction import DictVectorizer as DV
encoder = DV(sparse = False)
X_train_cat_oh = encoder.fit_transform(X_train[categorical_cols].T.to_dict().values())
X_test_cat_oh = encoder.transform(X_test[categorical_cols].T.to_dict().values())

In [283]:
optimizer.fit(X_train_cat_oh, y_train_bin)
optimizer_rf.fit(X_train_cat_oh, y_train_bin)
print ('Лучшие параметры для модели LogisticRegression для всех данных: {}'.format(optimizer.best_params_))
print ('Лучшие параметры для ансамбля RandomForestClassifier для всех данных: {}'.format(optimizer_rf.best_params_))

Лучшие параметры для модели LogisticRegression для всех данных: {'C': 1, 'penalty': 'l2'}
Лучшие параметры для ансамбля RandomForestClassifier для всех данных: {'max_depth': 14, 'min_samples_split': 8}


In [284]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score 
print('Доля верных ответов согласно метрике AUC ROC для модели LogisticRegression: {} '.format(roc_auc_score(y_test_bin,optimizer.best_estimator_.predict_proba(X_test_cat_oh)[:, 1])))
print('Доля верных ответов согласно метрике AUC ROC для ансамбля RandomForestClassifier: {} '.format(roc_auc_score(y_test_bin,optimizer_rf.best_estimator_.predict_proba(X_test_cat_oh)[:, 1])))
print('Доля верных ответов согласно метрике precision для модели LogisticRegression: {} '.format(precision_score(y_test_bin,optimizer.best_estimator_.predict(X_test_cat_oh))))
print('Доля верных ответов согласно метрике precision для ансамбля RandomForestClassifier: {} '.format(precision_score(y_test_bin,optimizer_rf.best_estimator_.predict(X_test_cat_oh))))
print('Доля верных ответов согласно метрике recall для модели LogisticRegression: {} '.format(recall_score(y_test_bin,optimizer.best_estimator_.predict(X_test_cat_oh))))
print('Доля верных ответов согласно метрике recall для ансамбля RandomForestClassifier: {} '.format(recall_score(y_test_bin,optimizer_rf.best_estimator_.predict(X_test_cat_oh))))
print('Доля верных ответов согласно метрике accuracy для модели LogisticRegression: {} '.format(accuracy_score(y_test_bin,optimizer.best_estimator_.predict(X_test_cat_oh))))
print('Доля верных ответов согласно метрике accuracy для ансамбля RandomForestClassifier: {} '.format(accuracy_score(y_test_bin,optimizer_rf.best_estimator_.predict(X_test_cat_oh))))
print('Доля верных ответов согласно метрике f1-score для модели LogisticRegression: {} '.format(f1_score(y_test_bin,optimizer.best_estimator_.predict(X_test_cat_oh))))
print('Доля верных ответов согласно метрике f1-score для ансамбля RandomForestClassifier: {} '.format(f1_score(y_test_bin,optimizer_rf.best_estimator_.predict(X_test_cat_oh))))

Доля верных ответов согласно метрике AUC ROC для модели LogisticRegression: 0.8784433918571057 
Доля верных ответов согласно метрике AUC ROC для ансамбля RandomForestClassifier: 0.8762816254507839 
Доля верных ответов согласно метрике precision для модели LogisticRegression: 0.8664764193451487 
Доля верных ответов согласно метрике precision для ансамбля RandomForestClassifier: 0.8585023976392475 
Доля верных ответов согласно метрике recall для модели LogisticRegression: 0.9278648974668275 
Доля верных ответов согласно метрике recall для ансамбля RandomForestClassifier: 0.935826296743064 
Доля верных ответов согласно метрике accuracy для модели LogisticRegression: 0.8356980529451508 
Доля верных ответов согласно метрике accuracy для ансамбля RandomForestClassifier: 0.8331797801117867 
Доля верных ответов согласно метрике f1-score для модели LogisticRegression: 0.8961205390081938 
Доля верных ответов согласно метрике f1-score для ансамбля RandomForestClassifier: 0.8954982685648327 


Как видно, качество логистической регрессии ,по сравнению с прошлым пунктом, заметно улучшилось. Это связано с тем, что LabelEncoder плохо работает с линейными моделями. Метрики близости объектов для LabelEncoder считают преобразованные числа для категориальных признаков весами. Т.е в следствие этого чем признак в таблице дальше от другого признака по порядку, тем больше разница между этими весами, а значит и больше метрика.